# QSAR Model Development  

## Overview  

This notebook performs the development and evaluation of quantitative structure–activity relationship (QSAR) models using the generated molecular descriptor dataset.

The descriptor matrix is processed and partitioned into training and testing subsets prior to model construction. Machine learning regression algorithms are applied to establish relationships between molecular features and biological activity.

Model performance is evaluated using standard regression metrics, including the coefficient of determination (R²) and root mean squared error (RMSE). Cross-validation is employed to assess model robustness and generalisability.

This stage completes the computational pipeline by translating molecular descriptors into predictive models suitable for virtual screening and structure–activity analysis.

**Step 1: Import Required Libraries**

This section imports the essential libraries for data handling, feature preprocessing, model training, evaluation, and visualisation within the QSAR modelling workflow.

In [ ]:
# === Core scientific libraries ===
import pandas as pd
import numpy as np
from numpy.linalg import pinv  # Used in leverage/applicability domain calculations

# === Visualisation ===
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D

# === Scikit-learn: preprocessing & feature selection ===
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold, RFECV

# === Scikit-learn: model training & evaluation ===
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import (
    KFold,
    cross_val_predict,
    RandomizedSearchCV,
    train_test_split
)
from sklearn.metrics import r2_score, mean_squared_error

### **Step 2: Load Required Input Files**

This step loads the curated bioactivity dataset together with the descriptor matrices corresponding to the selected molecular fingerprint. These files are required for model training, validation, and prediction.

**Required files:**
- `<Fingerprint>.csv` — Descriptor matrix for training compounds  
- `<Fingerprint>_Analogues.csv` — Descriptor matrix for external prediction set  
- `bioactivity_dataset_curated.csv` — Curated bioactivity dataset  

Ensure that the filenames correspond to the selected fingerprint type before proceeding.

In [ ]:
from pathlib import Path
import pandas as pd

# Define fingerprint type (must match descriptor file prefix)
fingerprint = "Substructure"  # Adjust as needed

DATA_DIR = Path("../data/processed")

# Define file paths
train_descriptor_file = DATA_DIR / f"{fingerprint}.csv"
external_descriptor_file = DATA_DIR / f"{fingerprint}_Analogues.csv"
bioactivity_file = DATA_DIR / "bioactivity_dataset_curated.csv"

# Validate existence
for file in [train_descriptor_file, external_descriptor_file, bioactivity_file]:
    assert file.exists(), f"Missing required file: {file}"

# Load datasets
X_train_descriptors = pd.read_csv(train_descriptor_file)
X_external_descriptors = pd.read_csv(external_descriptor_file)
bioactivity_df = pd.read_csv(bioactivity_file)

print("All required datasets loaded successfully.")
print(f"Training descriptor shape: {X_train_descriptors.shape}")
print(f"External descriptor shape: {X_external_descriptors.shape}")
print(f"Bioactivity dataset shape: {bioactivity_df.shape}")

**Step 3: Merge Descriptor and Bioactivity Data**

The descriptor matrix is merged with the curated bioactivity dataset using the compound identifier.  
The combined dataset is then partitioned into internal (training) and external (test) sets prior to model development.

In [ ]:
# Define fingerprint type (must match descriptor file prefix)
fingerprint = "AtomPairs2DCount"  # Adjust as needed

# Load descriptor and bioactivity datasets
desc_all = pd.read_csv(f"{fingerprint}.csv")            # Training descriptors
bioactivity = pd.read_csv("bioactivity_dataset_curated.csv")  # Curated activity data
tq_desc = pd.read_csv(f"{fingerprint}_Analogues.csv")   # External prediction descriptors

# Merge descriptors with bioactivity on compound ID
df = pd.merge(desc_all, bioactivity, on="ID")

print(f"Merged dataset shape: {df.shape}")

# Split dataset (80/20 ratio)
int_set, ext_set = train_test_split(df, test_size=0.2, random_state=42)

# Separate descriptors and activity
int_desc = int_set.drop(columns=["ID", "pIC50"]).reset_index(drop=True)
ext_desc = ext_set.drop(columns=["ID", "pIC50"]).reset_index(drop=True)
tq_desc  = tq_desc.drop(columns="ID").reset_index(drop=True)

# Keep numeric descriptor columns only
descriptor_cols = int_desc.select_dtypes(include=[np.number]).columns
int_desc = int_desc[descriptor_cols]
ext_desc = ext_desc[descriptor_cols]

# Define response variables
y_int = int_set["pIC50"].reset_index(drop=True)
y_ext = ext_set["pIC50"].reset_index(drop=True)

print("Internal descriptor shape:", int_desc.shape)
print("External descriptor shape:", ext_desc.shape)
print("Internal activity length:", len(y_int))
print("External activity length:", len(y_ext))

**Step 4: Descriptor Preprocessing and Feature Selection**

Descriptors are cleaned, standardised, and filtered to remove redundant or non-informative variables.  

Feature selection is performed in multiple stages:
1. Removal of constant and missing-value descriptors  
2. Standard scaling (fit on internal set only)  
3. High-correlation filtering  
4. Variance threshold filtering  
5. Activity-correlation filtering  
6. Hybrid feature selection (Random Forest ranking + RFECV)
7. Final train-test split

All preprocessing steps are fitted exclusively on the internal training set to prevent data leakage.

**1. Defensive cleaning**

In [ ]:
int0 = (
    int_desc
    .dropna(axis=1, how="all")
    .loc[:, lambda df: df.std() > 0]
    .dropna(axis=0)
)

clean_cols = int0.columns
print(f"After defensive cleaning → shape: {int0.shape}")

**2. Scale all sets with same scaler (fit on internal only)**

In [ ]:
scaler = StandardScaler().fit(int0)

X_int_scaled = pd.DataFrame(
    scaler.transform(int0),
    columns=clean_cols,
    index=int0.index
)

X_ext_scaled = pd.DataFrame(
    scaler.transform(ext_desc[clean_cols]),
    columns=clean_cols,
    index=ext_desc.index
)

X_tq_scaled = pd.DataFrame(
    scaler.transform(tq_desc[clean_cols]),
    columns=clean_cols,
    index=tq_desc.index
)

**3. Drop high-correlation descriptors (>|0.95|)**

In [ ]:
corr = X_int_scaled.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
drop1 = [c for c in upper.columns if any(upper[c] > 0.95)]

X_int_filt = X_int_scaled.drop(columns=drop1)
X_ext_filt = X_ext_scaled.drop(columns=drop1)
X_tq_filt  = X_tq_scaled.drop(columns=drop1)

print(f"After high-correlation filter (>|0.95|): {X_int_filt.shape[1]} descriptors")

**4. Variance filter (threshold = 0.01)**

In [ ]:
vt = VarianceThreshold(threshold=0.01).fit(X_int_filt)

keep_cols = X_int_filt.columns[vt.get_support()]

X_int_var = pd.DataFrame(
    vt.transform(X_int_filt),
    columns=keep_cols,
    index=X_int_filt.index
)

X_ext_var = pd.DataFrame(
    vt.transform(X_ext_filt),
    columns=keep_cols,
    index=X_ext_filt.index
)

X_tq_var = pd.DataFrame(
    vt.transform(X_tq_filt),
    columns=keep_cols,
    index=X_tq_filt.index
)

print(f"After VarianceThreshold(0.01): {X_int_var.shape[1]} descriptors")

**5. Drop descriptors highly correlated with activity (|r| > 0.7)**

In [ ]:
r_with_y = X_int_var.apply(lambda c: c.corr(y_int))
drop_cols = r_with_y[r_with_y.abs() > 0.7].index.tolist()

X_int_red = X_int_var.drop(columns=drop_cols, errors="ignore")
X_ext_red = X_ext_var.drop(columns=drop_cols, errors="ignore")
X_tq_red  = X_tq_var.drop(columns=drop_cols, errors="ignore")

print(f"After activity-correlation filter (|r|>0.7): {X_int_red.shape[1]} descriptors")

**6. Hybrid feature selection: RF ranking → RFECV**

In [ ]:
# ----- Stage 1: Random Forest ranking (Top-K selection) -----
top_k = min(50, X_int_red.shape[1])

rf_rank = RandomForestRegressor(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)

rf_rank.fit(X_int_red, y_int)

importances = rf_rank.feature_importances_
order = np.argsort(importances)[::-1]
topK_idx = order[:top_k]
topK_cols = X_int_red.columns[topK_idx]

X_int_topK = X_int_red[topK_cols].copy()

print(f"Stage 1 → Top-{top_k} preselected features")

# ----- Stage 2: RFECV on Top-K -----
cv10 = KFold(n_splits=10, shuffle=True, random_state=42)

rf_base = RandomForestRegressor(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)

rfecv = RFECV(
    estimator=rf_base,
    step=3,
    cv=cv10,
    scoring="neg_mean_squared_error",
    n_jobs=-1,
    min_features_to_select=30
)

rfecv.fit(X_int_topK, y_int)

sel_mask = rfecv.support_
sel_cols = X_int_topK.columns[sel_mask]

print(f"RFECV selected {len(sel_cols)} optimal features")

# Final selected feature matrices
X_int_sel = X_int_red[sel_cols].copy()
X_ext_sel = X_ext_red[sel_cols].copy()
X_tq_sel  = X_tq_red[sel_cols].copy()

**7. Final train-test split**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_int_sel,
    y_int,
    test_size=0.2,
    random_state=42
)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"Training activity length: {y_train.shape}")
print(f"Test activity length: {y_test.shape}")

**Step 5: Hyperparameter Optimisation and Model Evaluation**

Randomised hyperparameter search is performed to optimise the Random Forest regression model.  

Model performance is evaluated using:
- Coefficient of determination (R²)
- Root Mean Squared Error (RMSE)

Performance is assessed on:
1. Training set  
2. Independent test set  
3. Cross-validation predictions (internal robustness assessment)

In [ ]:
# === Hyperparameter Tuning ===

# Define parameter search space
param_dist = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 20, 30],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', 0.5],
    'bootstrap': [True, False]
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=50,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

# Fit on training data
random_search.fit(X_train, y_train)

# Retrieve best model
best_model = random_search.best_estimator_

print("Best Random Forest parameters:")
print(random_search.best_params_)

# === 1. Training Set Performance ===
y_pred_train = best_model.predict(X_train)
r2_train = r2_score(y_train, y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))

# === 2. Test Set Performance ===
y_pred_test = best_model.predict(X_test)
r2_test = r2_score(y_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

# === 3. Cross-Validation Performance (Internal Set) ===
cv10 = KFold(n_splits=10, shuffle=True, random_state=42)
y_cv_pred = cross_val_predict(best_model, X_int_sel, y_int, cv=cv10)

r2_cv = r2_score(y_int, y_cv_pred)
rmse_cv = np.sqrt(mean_squared_error(y_int, y_cv_pred))

# === Report Results ===
print("\nModel Performance Summary")
print("--------------------------")
print(f"Training Set   → R²: {r2_train:.3f} | RMSE: {rmse_train:.3f}")
print(f"Test Set       → R²: {r2_test:.3f} | RMSE: {rmse_test:.3f}")
print(f"Cross-Validation (10-fold) → R²: {r2_cv:.3f} | RMSE: {rmse_cv:.3f}")

**Step 6: Independent 10-Fold Cross-Validation (Q²CV)**

A manual 10-fold cross-validation procedure is performed using the optimised Random Forest model.  

For each fold:
- The model is trained on 90% of the internal data  
- Predictions are generated for the remaining 10%  
- R² and RMSE are computed  

The mean R² across folds is reported as Q²CV, representing internal predictive robustness.

In [ ]:
# === 10-Fold Cross-Validation (Independent Evaluation) ===

kf = KFold(n_splits=10, shuffle=True, random_state=42)

rmse_scores = []
r2_scores = []

for train_idx, val_idx in kf.split(X_int_sel):

    X_fold_train = X_int_sel.iloc[train_idx]
    X_fold_val   = X_int_sel.iloc[val_idx]

    y_fold_train = y_int.iloc[train_idx]
    y_fold_val   = y_int.iloc[val_idx]

    # Re-instantiate model using best hyperparameters
    model = RandomForestRegressor(
        **random_search.best_params_,
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_fold_train, y_fold_train)

    y_fold_pred = model.predict(X_fold_val)

    rmse = np.sqrt(mean_squared_error(y_fold_val, y_fold_pred))
    r2   = r2_score(y_fold_val, y_fold_pred)

    rmse_scores.append(rmse)
    r2_scores.append(r2)

# Compute mean Q²CV and RMSECV
q2_cv   = np.mean(r2_scores)
rmse_cv = np.mean(rmse_scores)

print("\n10-Fold Cross-Validation Results")
print("----------------------------------")
print(f"Q²CV (mean R²): {q2_cv:.3f}")
print(f"RMSECV (mean RMSE): {rmse_cv:.3f}")

**Step 7: External Prediction and Performance Summary**

The optimised Random Forest model is applied to the external validation set.  

Performance is evaluated using:
- R² (external test set)
- RMSE (external test set)
- Q²Ext (external predictive coefficient)

A consolidated performance table summarises all validation metrics.

In [ ]:
# === External Predictions ===
y_ext_pred = best_model.predict(X_ext_sel)

R2_Ext = r2_score(y_ext, y_ext_pred)
RMSE_Ext = np.sqrt(mean_squared_error(y_ext, y_ext_pred))

# Q²Ext calculation (external predictive squared correlation)
Q2_Ext = 1 - (
    ((y_ext - y_ext_pred) ** 2).sum() /
    ((y_ext - y_train.mean()) ** 2).sum()
)

print("\nExternal Validation Results")
print("----------------------------")
print(f"R²_Ext  : {R2_Ext:.3f}")
print(f"RMSE_Ext: {RMSE_Ext:.3f}")
print(f"Q²_Ext  : {Q2_Ext:.3f}")

# === Performance Summary Table ===
performance_metrics = pd.DataFrame({
    "Model": [fingerprint],
    "R²_Train": [r2_train],
    "RMSE_Train": [rmse_train],
    "R²_Test": [r2_test],
    "RMSE_Test": [rmse_test],
    "Q²_CV": [q2_cv],
    "RMSE_CV": [rmse_cv],
    "Q²_Ext": [Q2_Ext],
    "RMSE_Ext": [RMSE_Ext],
    "Δ(R²_Test − Q²_CV)": [r2_test - q2_cv],
    "Δ(R²_Test − Q²_Ext)": [r2_test - Q2_Ext],
    "Selected_Descriptors": [len(sel_cols)]
}).round(3)

performance_metrics

**Step 8: Test Set Predictive Coefficient (Q²Test)**

The predictive squared correlation coefficient (Q²Test) is calculated for the independent test set using the training set mean as reference.

In [ ]:
# === Q²Test Calculation (Independent Test Set Predictivity) ===
Q2_Test = 1 - (
    ((y_test - y_pred_test) ** 2).sum() /
    ((y_test - y_train.mean()) ** 2).sum()
)

print(f"Q²Test (independent test set): {Q2_Test:.3f}")

**Step 9: Y-Randomisation (Y-Scrambling) Test**

To assess the possibility of chance correlation, the response variable (activity) is randomly shuffled and the modelling procedure is repeated multiple times.

If the model is statistically robust, the resulting R² and Q² values from randomised models should be significantly lower than those obtained from the original model.

In [ ]:
# === Y-Randomisation Test ===

r2_randomised = []
q2_randomised = []
n_iterations = 100

for seed in range(n_iterations):

    # Shuffle activity values
    y_shuffled = y_int.sample(frac=1.0, random_state=seed).reset_index(drop=True)

    # Train-test split
    X_train_rand, X_test_rand, y_train_rand, y_test_rand = train_test_split(
        X_int_sel,
        y_shuffled,
        test_size=0.2,
        random_state=42
    )

    # Use optimised hyperparameters
    model_rand = RandomForestRegressor(
        **random_search.best_params_,
        random_state=42,
        n_jobs=-1
    )

    model_rand.fit(X_train_rand, y_train_rand)

    y_pred_rand = model_rand.predict(X_test_rand)

    r2_rand = r2_score(y_test_rand, y_pred_rand)

    q2_rand = 1 - (
        ((y_test_rand - y_pred_rand) ** 2).sum() /
        ((y_test_rand - y_train_rand.mean()) ** 2).sum()
    )

    r2_randomised.append(r2_rand)
    q2_randomised.append(q2_rand)

# === Summary Statistics ===
mean_r2_rand = np.mean(r2_randomised)
mean_q2_rand = np.mean(q2_randomised)

print("\nY-Randomisation Results")
print("------------------------")
print(f"Mean R² (randomised): {mean_r2_rand:.3f}")
print(f"Mean Q² (randomised): {mean_q2_rand:.3f}")
print(f"Original R²_Test: {r2_test:.3f}")
print(f"Original Q²_Test: {Q2_Test:.3f}")

**Step 10: Model Performance Visualisation**

Two validation plots are generated:

(a) Predicted vs Experimental pIC₅₀ for internal and external sets  
(b) Y-randomisation validation (R² vs Q² comparison)

Figures are exported in high resolution for reporting purposes.

In [ ]:
# ---------- Matplotlib Configuration ----------
mpl.rcParams.update({
    "text.usetex": False,
    "font.family": "STIXGeneral",
    "mathtext.fontset": "stix",
    "mathtext.default": "regular",
})

# ---------- Create Figure ----------
fig, axes = plt.subplots(1, 2, figsize=(15, 7))

# =======================
# (a) Predicted vs Experimental
# =======================
ax1 = axes[0]

ax1.scatter(
    y_test, y_pred_test,
    c="#2ca02c",
    label="Internal Test Set",
    edgecolor="black",
    s=60,
    alpha=0.7
)

ax1.scatter(
    y_ext, y_ext_pred,
    c="#d62728",
    label="External Set",
    edgecolor="black",
    s=60,
    alpha=0.8
)

min_val = min(
    y_test.min(), y_ext.min(),
    y_pred_test.min(), y_ext_pred.min()
)

max_val = max(
    y_test.max(), y_ext.max(),
    y_pred_test.max(), y_ext_pred.max()
)

# Ideal prediction line
ax1.plot([min_val, max_val], [min_val, max_val],
         linestyle="--", linewidth=2, label="Ideal Prediction")

ax1.set_xlabel(r'$\mathbf{Experimental\ pIC}_{50}$', fontsize=18)
ax1.set_ylabel(r'$\mathbf{Predicted\ pIC}_{50}$', fontsize=18)

ax1.set_xlim(min_val - 0.2, max_val + 0.2)
ax1.set_ylim(min_val - 0.2, max_val + 0.2)

ax1.legend(frameon=False, fontsize=14)
ax1.tick_params(axis='both', direction='in', length=6, width=1.5)
ax1.text(0.02, 0.98, '(a)', transform=ax1.transAxes,
         fontsize=15, va='top', fontweight='bold')

# =======================
# (b) Y-Randomisation Plot
# =======================
ax2 = axes[1]

ax2.scatter(
    q2_randomised,
    r2_randomised,
    color='#d62728',
    s=60,
    alpha=0.8,
    edgecolor='black',
    label='Y-Scrambled Models'
)

ax2.scatter(
    Q2_Test,
    r2_test,
    color='#2ca02c',
    s=90,
    edgecolor='black',
    label='Actual Model'
)

ax2.set_xlabel(r'$\mathbf{Q^2}$', fontsize=18)
ax2.set_ylabel(r'$\mathbf{R^2}$', fontsize=18)

ax2.set_xlim(-1.0, 1.5)
ax2.set_ylim(-1.0, 1.5)

ax2.legend(frameon=False, fontsize=14)
ax2.tick_params(axis='both', direction='in', length=6, width=1.5)
ax2.text(0.02, 0.98, '(b)', transform=ax2.transAxes,
         fontsize=15, va='top', fontweight='bold')

plt.tight_layout()

# ---------- Save Figures ----------
for ext in ["png", "svg", "pdf"]:
    fname = f"QSAR_Model_Validation.{ext}"
    fig.savefig(fname, dpi=600, bbox_inches='tight', facecolor='white')

plt.show()
plt.close(fig)

**Step 11: Applicability Domain Assessment (Williams Plot)**

The Williams plot is generated to evaluate the model's applicability domain (AD).

Leverage values identify structurally influential compounds, while standardised residuals detect response outliers.

Compounds are considered:
- Influential if leverage > h*
- Outliers if |standardised residual| > 3

The leverage threshold is defined as:

\[
h^* = \frac{3(p+1)}{n}
\]

where *p* is the number of selected descriptors and *n* is the number of training compounds.

In [ ]:
# ---------- Matplotlib Configuration ----------
mpl.rcParams.update({
    "text.usetex": False,
    "font.family": "STIXGeneral",
    "mathtext.fontset": "stix",
    "mathtext.default": "regular",
})

# ---------- Keep IDs aligned ----------
train_ids = int_set["ID"].reset_index(drop=True).astype(str)
test_ids  = ext_set["ID"].reset_index(drop=True).astype(str)

# ---------- Prepare NumPy matrices ----------
X_train_np = X_train.to_numpy()
X_test_np  = X_test.to_numpy()

# === Step 1: Leverage Calculation ===
XT_X_inv = np.linalg.pinv(X_train_np.T @ X_train_np)

H = X_train_np @ XT_X_inv @ X_train_np.T
leverage_train = np.diag(H)

leverage_test = np.array([x @ XT_X_inv @ x.T for x in X_test_np])

# === Step 2: Leverage Threshold ===
n, p = X_train_np.shape
h_star = 3 * (p + 1) / n

print(f"Leverage threshold (h*): {h_star:.4f}")

# === Step 3: Standardised Residuals ===
residuals_train = y_pred_train - y_train
residuals_test  = y_pred_test - y_test

std_train = np.std(residuals_train)
std_test  = np.std(residuals_test)

standardised_residuals_train = residuals_train / (std_train if std_train != 0 else 1.0)
standardised_residuals_test  = residuals_test  / (std_test if std_test != 0 else 1.0)

# === Step 4: AD Statistics ===
outliers_train = np.where(np.abs(standardised_residuals_train) > 3)[0]
influential_train = np.where(leverage_train > h_star)[0]

outliers_test = np.where(np.abs(standardised_residuals_test) > 3)[0]
influential_test = np.where(leverage_test > h_star)[0]

print("\nInternal Set:")
print(f"Outliers (|residual| > 3): {len(outliers_train)}")
print(f"Influential (leverage > h*): {len(influential_train)}")

print("\nExternal Set:")
print(f"Outliers (|residual| > 3): {len(outliers_test)}")
print(f"Influential (leverage > h*): {len(influential_test)}")

# === Step 5: Williams Plot ===
fig, ax = plt.subplots(figsize=(10, 6))

ax.scatter(
    leverage_train,
    standardised_residuals_train,
    c='#2ca02c',
    edgecolor='black',
    s=60,
    alpha=0.8,
    label='Internal set'
)

ax.scatter(
    leverage_test,
    standardised_residuals_test,
    c='#d62728',
    edgecolor='black',
    s=60,
    alpha=0.8,
    label='External set'
)

# Threshold lines
ax.axhline(3, color='blue', linewidth=1.5)
ax.axhline(-3, color='blue', linewidth=1.5)
ax.axvline(h_star, color='blue', linestyle='--', linewidth=1.5)

ax.set_xlabel(r'$\mathbf{Leverage}$', fontsize=18)
ax.set_ylabel(r'$\mathbf{Standardised\ Residuals}$', fontsize=18)

ax.set_ylim(-5, 5)
ax.set_xlim(left=0)

ax.tick_params(axis='both', direction='in', length=6, width=1.5)
ax.legend(frameon=False, fontsize=12)

for spine in ax.spines.values():
    spine.set_linewidth(1.5)

plt.tight_layout()

# ---------- Save Figure ----------
for ext in ["png", "svg", "pdf"]:
    fname = f"WilliamsPlot_{fingerprint}.{ext}"
    fig.savefig(fname, dpi=600, bbox_inches='tight', facecolor='white')

plt.show()
plt.close(fig)

**Step 12: External Library Prediction and Activity Classification**

The trained QSAR model is applied to an external analogue library.  

Predicted pIC₅₀ values are generated and compounds are categorised into:
- Active (pIC₅₀ ≥ 6)
- Intermediate (5 ≤ pIC₅₀ < 6)
- Inactive (pIC₅₀ < 5)

Predictions are appended to the original library file and exported for downstream analysis.

In [ ]:
**Distribution of TQ Analogues by Applicability Domain and Activity Class**
# ---------- Bubble Plot of AD vs Activity Class ----------
from matplotlib.colors import LinearSegmentedColormap

# ---------- Fonts ----------
mpl.rcParams.update({
    "text.usetex": False,
    "font.family": "STIXGeneral",
    "mathtext.fontset": "stix",
    "mathtext.default": "regular",
})

# ---------- Load dataset ----------
file_path = 'Thymoquinone_Analogues_library.csv'
counts = (
    pd.read_csv(file_path)
    .groupby(['AD_Flag', 'Activity_Class'])
    .size()
    .reset_index(name='Count')
)

# ---------- Map categories to numeric axes ----------
x_map = {'Inside AD': 0, 'Outside AD': 1}
y_map = {'Inactive': 0, 'Intermediate': 1, 'Active': 2}
counts['x'] = counts['AD_Flag'].map(x_map)
counts['y'] = counts['Activity_Class'].map(y_map)

# ---------- Define bubble sizes ----------
bubble_sizes = counts['Count'] * 20 + 200  # scale factor + base size

# ---------- Custom colormap ----------
cmap = LinearSegmentedColormap.from_list('green_blue_red', ['green', 'blue', 'red'])

# ---------- Create plot ----------
fig, ax = plt.subplots(figsize=(8, 6))
scatter = ax.scatter(
    counts['x'], counts['y'],
    s=bubble_sizes,
    c=counts['Count'],
    cmap=cmap,
    alpha=0.7,
    edgecolors='black',
    linewidth=1.2
)

# ---------- Annotate bubbles ----------
for _, row in counts.iterrows():
    ax.text(
        row['x'], row['y'], int(row['Count']),
        ha='center', va='center', fontsize=12, color='white'
    )

# ---------- Axes limits and labels ----------
ax.set_xlim(-0.5, 1.5)
ax.set_ylim(-0.5, 2.5)
ax.set_xticks([0, 1])
ax.set_xticklabels(['Inside AD', 'Outside AD'], fontsize=16)
ax.set_yticks([0, 1, 2])
ax.set_yticklabels(['Inactive', 'Intermediate', 'Active'], fontsize=16)
ax.set_xlabel(r'$\mathbf{Applicability\ Domain\ Status}$', fontsize=16)
ax.set_ylabel(r'$\mathbf{Activity\ Class}$', fontsize=16)

# ---------- Colorbar ----------
cbar = fig.colorbar(scatter, ax=ax, pad=0.02)
cbar.set_label('Number of Compounds', fontsize=16, fontweight='bold')
cbar.ax.tick_params(labelsize=10)

# ---------- Styling ----------
ax.grid(False)
for spine in ax.spines.values():
    spine.set_edgecolor('black')
    spine.set_linewidth(2.0)

fig.tight_layout()

# ---------- Save & Auto-Download ----------
for ext in ["png", "svg", "pdf"]:
    fname = f"Bubble_AD_Activity.{ext}"
    fig.savefig(fname, dpi=600, bbox_inches='tight', facecolor='white')
    files.download(fname)

# ---------- Display and close ----------
plt.show()
plt.close(fig)

**Step 13: Applicability Domain Assessment for External Library**

Leverage values are computed for the external analogue library using the training-set hat matrix.  

Compounds are classified as:
- Inside AD (leverage ≤ h*)
- Outside AD (leverage > h*)

The leverage values and AD flags are appended to the prediction output file.

In [ ]:
# === Applicability Domain for External Library ===

# Convert selected descriptor matrix to NumPy
X_tq_np = X_tq_sel.to_numpy()

# Compute leverage values
leverage_tq = np.array([x @ XT_X_inv @ x.T for x in X_tq_np])

# Assign AD classification
ad_flag = np.where(leverage_tq > h_star, "Outside AD", "Inside AD")

# Summary statistics
n_inside = np.sum(ad_flag == "Inside AD")
n_outside = np.sum(ad_flag == "Outside AD")

print("\nApplicability Domain Summary")
print("-----------------------------")
print(f"Inside AD : {n_inside}")
print(f"Outside AD: {n_outside}")

# === Append AD Results to Prediction File ===
from pathlib import Path

DATA_DIR = Path("../data/processed")
prediction_file = DATA_DIR / f"{fingerprint}_Analogues_Predictions.csv"

assert prediction_file.exists(), f"Prediction file not found: {prediction_file}"

final_df = pd.read_csv(prediction_file)

# Sanity check alignment
if len(final_df) != len(leverage_tq):
    raise ValueError(
        f"Row mismatch: prediction file has {len(final_df)} rows "
        f"but AD array has {len(leverage_tq)} rows."
    )

# Add AD columns
final_df["Leverage"] = leverage_tq
final_df["AD_Flag"] = ad_flag

# Save updated file
output_file = DATA_DIR / f"{fingerprint}_Analogues_Predictions_AD.csv"
final_df.to_csv(output_file, index=False)

print(f"\nAD results saved to: {output_file}")

**Step 14: Distribution of External Library by Applicability Domain and Activity Class**

A bubble plot is generated to visualise the distribution of predicted compounds according to:

- Applicability Domain status (Inside / Outside AD)
- Activity classification (Inactive / Intermediate / Active)

Bubble size and colour reflect the number of compounds in each category.

In [ ]:
# ---------- Matplotlib Configuration ----------
from matplotlib.colors import LinearSegmentedColormap
from pathlib import Path

mpl.rcParams.update({
    "text.usetex": False,
    "font.family": "STIXGeneral",
    "mathtext.fontset": "stix",
    "mathtext.default": "regular",
})

# ---------- Load Prediction + AD Dataset ----------
DATA_DIR = Path("../data/processed")
file_path = DATA_DIR / f"{fingerprint}_Analogues_Predictions_AD.csv"

assert file_path.exists(), f"File not found: {file_path}"

counts = (
    pd.read_csv(file_path)
    .groupby(['AD_Flag', 'Activity_Class'])
    .size()
    .reset_index(name='Count')
)

# ---------- Map Categories to Axes ----------
x_map = {'Inside AD': 0, 'Outside AD': 1}
y_map = {'Inactive': 0, 'Intermediate': 1, 'Active': 2}

counts['x'] = counts['AD_Flag'].map(x_map)
counts['y'] = counts['Activity_Class'].map(y_map)

# ---------- Bubble Sizes ----------
bubble_sizes = counts['Count'] * 20 + 200

# ---------- Custom Colormap ----------
cmap = LinearSegmentedColormap.from_list(
    'green_blue_red',
    ['green', 'blue', 'red']
)

# ---------- Create Plot ----------
fig, ax = plt.subplots(figsize=(8, 6))

scatter = ax.scatter(
    counts['x'],
    counts['y'],
    s=bubble_sizes,
    c=counts['Count'],
    cmap=cmap,
    alpha=0.75,
    edgecolors='black',
    linewidth=1.2
)

# ---------- Annotate Bubbles ----------
for _, row in counts.iterrows():
    ax.text(
        row['x'],
        row['y'],
        int(row['Count']),
        ha='center',
        va='center',
        fontsize=11,
        color='white'
    )

# ---------- Axes Formatting ----------
ax.set_xlim(-0.5, 1.5)
ax.set_ylim(-0.5, 2.5)

ax.set_xticks([0, 1])
ax.set_xticklabels(['Inside AD', 'Outside AD'], fontsize=14)

ax.set_yticks([0, 1, 2])
ax.set_yticklabels(['Inactive', 'Intermediate', 'Active'], fontsize=14)

ax.set_xlabel(r'$\mathbf{Applicability\ Domain}$', fontsize=16)
ax.set_ylabel(r'$\mathbf{Activity\ Class}$', fontsize=16)

# ---------- Colorbar ----------
cbar = fig.colorbar(scatter, ax=ax, pad=0.02)
cbar.set_label('Number of Compounds', fontsize=14)
cbar.ax.tick_params(labelsize=10)

# ---------- Styling ----------
ax.grid(False)
for spine in ax.spines.values():
    spine.set_linewidth(1.5)

plt.tight_layout()

# ---------- Save Figure ----------
for ext in ["png", "svg", "pdf"]:
    fname = DATA_DIR / f"AD_Activity_Distribution_{fingerprint}.{ext}"
    fig.savefig(fname, dpi=600, bbox_inches='tight', facecolor='white')

plt.show()
plt.close(fig)